```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de Datos no Estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2020
Profesor: Facundo Santiago, Javier Ignacio Garcia Fronti
```

# Exploración de un modelo entrenado con Word2Vec para el idioma español

En esta sección, exploraremos las cualidades de las representaciones vectoriales generadas por el algoritmo Word2Vec para el idioma español. Este notebook requiere tener instaladas las siguientes librerias:
- gensim
- numpy
- tensorflow

In [12]:
import sys, os, io, pathlib
import numpy as np

from gensim.models import KeyedVectors
from gensim.test.utils import datapath

import warnings
warnings.filterwarnings('ignore')

## Cargamos un modelo pre-entrenado en Español

Utilizaremos un modelo pre-entrenado de Word2Vec que fué entrenado con un corpus en español (Spanish CoNLL17 corpus). Este modelo se entrenó trabajando con una ventana de 10 palabras, con el algoritmo Continuous bag of words y un negative-sampling de 5 palabras. Este modelo está almacenado en formato binario, por lo que lo cargamos utilizando el objeto KeyedVectors y el metodo load_word2vec_format

Descargamos el modelo

In [ ]:
!mkdir -p ./Models/Word2Vec
!wget -N https://santiagxf.blob.core.windows.net/public/Word2Vec/model.bin --directory-prefix ./Models/Word2Vec

Referencia: NLPL Word Vectors Repository (http://vectors.nlpl.eu/repository/), version 2.0

Cargamos el embedding (este embedding es bastante grande)

In [13]:
path = os.path.abspath('Models/Word2Vec/model.bin')
embeddings = KeyedVectors.load_word2vec_format(datapath(path), binary=True)

Podemos objetener algunas propiedades del modelo

In [15]:
print ("El tamaño del vocabulario es:", len(embeddings.vocab))
print("El tamaño de los vectores es:", embeddings.vector_size)

El tamaño del vocabulario es: 2656057
El tamaño de los vectores es: 100


## Exploramos las representaciones
Podemos comenzar a indagar como lucen las representaciones vectoriales que se aprendieron utilizando el método most_similar. Este método nos devuelve las 10 primeras palabras más similares a la palabra que indicamos. La similaridad se computa utilizando cosine-similarity de las representaciones vectoriales de cada una de las palabras.

In [17]:
embeddings.most_similar("reina")

[('emperatriz', 0.7729766964912415),
 ('princesa', 0.7713199257850647),
 ('sofía', 0.7699436545372009),
 ('coronaciónla', 0.7649693489074707),
 ('sofíala', 0.7604084014892578),
 ('virreina', 0.7558672428131104),
 ('categoría:talavera', 0.7512167692184448),
 ('isabel', 0.7497586011886597),
 ('sofíael', 0.735364556312561),
 ('emperatríz', 0.7345834970474243)]

También podemos consultar cuales son las palabras más disimilares a una determinada palabra. Esto lo hacemos especificando el parametro negative dentro del método most_similar

In [18]:
embeddings.most_similar(negative="reina")

[('86222880', 0.37661516666412354),
 ('cze-', 0.3465925455093384),
 ('episodionumero=21', 0.32355785369873047),
 ('confederaciónuefa', 0.32355785369873047),
 ('borderisrael', 0.3164479732513428),
 ('categoría:syntermitinae', 0.3074568510055542),
 ('categoría:socialdemocracia', 0.2885240316390991),
 ('03.05.82', 0.2859065532684326),
 ('categoría:lujanenses', 0.28335657715797424),
 ('hansa-sur', 0.2776678204536438)]

## Analogias

El método anterior no es muy util en general, sin embargo nos permite introducir el concepto de aritmética sobre estos mismos vectores. Por ejemplo: Pordiamos consultar cual es la analogia de dos palabras. El siguiente ejemplo se leería así: A lo que Paris es a Francia, ¿cúal es la analogia de Madrid?. Esto se resolveria tomando "Francia", quitandole "Paris" y agregandole "Madrid"

<img src="Docs/word2vec-math.png" width="500" />

In [19]:
embeddings.most_similar(positive=["francia", "madrid"], negative=["paris"])[0][0]

'españa'

Estos nos dice que quizás este espacio vectorial no es solamente un espacio donde vectores que están cerca los unos de los otros en el espacio tienen significados similares, sino que en realidad capturan el significado en una forma más produnda. Concretamente, que hay "direcciones de significados" en el espacio donde uno de puede mover. Veamos algunos ejemplos:

En la dirección de "hacedor"
 - Conducir es a conductor lo que limpiar es a?

In [20]:
embeddings.most_similar(positive=["conductor", "limpiar"], negative=["conducir"])[0][0]

'trapeador'

En la dirección "bebidas"
 - El vino es a Francia lo que el whisky es a?

In [22]:
embeddings.most_similar(positive=["francia", "whisky"], negative=["vino"])[0][0]

'bélgica'

En la dirección "extremar"
 - Bueno es a genial lo que malo es a?

In [23]:
embeddings.most_similar(positive=["genial", "malo"], negative=["bueno"])[0][0]

'malísimo'

## Visualización del espacio contino de word2vec

El siguiente código utiliza un plugin de Tensorflow llamado Tensorboard que permite hacer una proyección en un espacio 3D (para luego hacer una proyección en un espacio 2D - el de la pantalla) de las representaciones de cada una de las palabras que están en el vocabulario. Esta visualización requiere tener instalado TensorFlow 2.0

**Importante:** Actualmente hay un problema con Google Colaboratory con la carga de Projector en Tensorboard. Al parecer es debido al modulo tfjs. Si quieren probar esta funcionalidad tendrán que ejecutar este codigo en su equipo local. Para hacer esto deberan descargar este notebook en su equipo con Python instalado y las librerias mensionadas en la parte superior. Adicionalmente:

!pip install tensorboard
!tensorboard --logdir ./logdir

In [4]:
import tensorflow as tf
from tqdm import tqdm
from tensorboard.plugins import projector

In [5]:
logdir = '/tmp/logdir'

In [6]:
!mkdir -p '/tmp/logdir'

In [10]:
with tf.device('/CPU:0'):
    weights = tf.Variable(embeddings.vectors, )
    checkpoint = tf.train.Checkpoint(embedding=weights)
    checkpoint.save(os.path.join(logdir, 'embedding.ckpt'))

El siguiente codigo genera los "labels" asociados al espacio vectorial que cargamos anteriormente. Pueden descargar estos labels ya preprocesados desde:

In [7]:
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/meta.tsv --directory-prefix '/tmp/logdir'

--2020-09-03 21:17:23--  https://santiagxf.blob.core.windows.net/public/Word2Vec/meta.tsv
Resolving santiagxf.blob.core.windows.net (santiagxf.blob.core.windows.net)... 52.239.220.32
Connecting to santiagxf.blob.core.windows.net (santiagxf.blob.core.windows.net)|52.239.220.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930085 (908K) [text/tab-separated-values]
Saving to: ‘/tmp/logdir/meta.tsv’

meta.tsv            100%[===================>] 908.29K  --.-KB/s    in 0.009s  

2020-09-03 21:17:23 (99.0 MB/s) - ‘/tmp/logdir/meta.tsv’ saved [930085/930085]



In [ ]:
with io.open(logdir + '/meta.tsv', 'w', encoding='utf-8') as metadata_file:
    with io.open(logdir + '/vecs.tsv', 'w', encoding='utf-8') as vectors_file:
        for index in tqdm(range(len(embeddings.index2word))):
            word = embeddings.index2word[index]
            vec = embeddings.vectors[index]
            metadata_file.write(word + "\n")
            vectors_file.write('\t'.join([str(x) for x in vec]) + "\n")

In [11]:
# Set up config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'meta.tsv'
projector.visualize_embeddings(logdir, config)

Cargamos la extensión de Google Colab Tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logdir